### Spam filtering
This assignment was carried out using a nice tutorial shared [here](https://www.kdnuggets.com/2017/03/email-spam-filtering-an-implementation-with-python-and-scikit-learn.html)


In [1]:
import os
import numpy as np
# to count number of instances in a list an return a dictionary
from collections import Counter
train_directory = "train-mails"

# make a dictionary of words in spam emails
def make_Dictionary(train_dr):
    emails_f = [f for f in os.listdir(train_dr)]
    all_words = []
    for mail in emails_f:
        with open(train_dr+'/'+mail) as m:
            for i, line in enumerate(m):
                if i==2: #body of email starts from the 3rd line
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    
    # removing single character and non-word
    list_to_remove = list(dictionary)
    for item in list_to_remove:
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary
spWordsdict = make_Dictionary(train_directory)

In [2]:
# construct word count vector
# each word count vector contains the frequency of 3000 words in the training file
def extract_features(mail_dir):
    files = [os.path.join(mail_dir,f) for f in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files), 3000))
    docID = 0
    for fil in files:
        with open(fil) as fi:
            for i, line in enumerate(fi):
                if i==2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(spWordsdict):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID, wordID] = words.count(word)
            docID +=1
    return features_matrix

In [3]:
XX = extract_features(train_directory)

In [4]:
from sklearn.naive_bayes import  MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import  SVC, NuSVC, LinearSVC

train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = XX

# Training SVM and Naive Bayes classifiers

model1 = MultinomialNB()
model2 = LinearSVC()
model1.fit(train_matrix, train_labels)
model2.fit(train_matrix, train_labels)

# Test the unseen mails for Spam
test_dir = 'test-mails'
test_matrix = extract_features(test_dir)
test_labels = np.zeros(260)
test_labels[130:260] = 1
result1 = model1.predict(test_matrix)
result2 = model2.predict(test_matrix)


In [5]:
from sklearn.metrics import  confusion_matrix
print(confusion_matrix(test_labels,result1))
print(confusion_matrix(test_labels,result2))
!ls

[[ 29 101]
 [ 32  98]]
[[59 71]
 [52 78]]
emailSample1.txt  ex6data2.mat	ex6.pdf		     spamSample2.txt  vocab.txt
emailSample2.txt  ex6data3.mat	ex6_spamEmail.ipynb  test-mails
ex6data1.mat	  ex6.ipynb	spamSample1.txt      train-mails
